In [1]:
import tensorflow as tf
from tensorflow.keras import Model
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

tf.random.set_seed(42)
type_train_ds = tf.data.experimental.load("/kaggle/input/cmi-tf-datasets/type_train_ds")
type_valid_ds = tf.data.experimental.load("/kaggle/input/cmi-tf-datasets/type_valid_ds")
gesture_train_ds = tf.data.experimental.load("/kaggle/input/cmi-tf-datasets/gesture_train_ds")
gesture_valid_ds = tf.data.experimental.load("/kaggle/input/cmi-tf-datasets/gesture_valid_ds")
# test_ds = tf.data.experimental.load("/kaggle/input/cmi-tf-datasets/test_ds")

2025-08-13 05:35:47.523890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755063347.721571      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755063347.776465      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1755063361.664257      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [2]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [3]:
print(len(type_train_ds))
print(len(type_valid_ds))
print(len(gesture_train_ds))
print(len(gesture_valid_ds))

# print(len(test_ds))

146
24
146
24


In [4]:
for i, (inputs, targets) in enumerate(type_train_ds):
    if i==0:
        print(inputs[0].shape)
        print(inputs[1].shape)
        print(inputs[2].shape)
        print(targets.shape)
        break

(64, 50, 8, 8, 5)
(64, 50, 19)
(64, 17)
(64, 1)


In [5]:
from tensorflow.keras.layers import Conv3D, Conv2D, Conv1D, BatchNormalization, TimeDistributed, InputLayer
from tensorflow.keras.layers import MaxPool3D, MaxPool2D, GlobalMaxPool3D, GlobalMaxPool2D, AveragePooling1D, Dropout
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Flatten, Dense, GaussianNoise
from tensorflow.keras.layers import ReLU, ELU, Masking
from tensorflow.keras.regularizers import L1, L2, L1L2
from tensorflow.keras import Sequential

class RNNModel(Model):
    def __init__(self, kernel_size3d=3, kernel_size1d=3, filters_3d=[16, 32], dropout=0.2, 
                 filters_1d=[16, 32], num_rnn_layers=1, rnn_hidden_size=32, mode="RNN", 
                 bidirectional=False, hidden_size=64, regularizer="l1", l1_penalty=0.1, l2_penalty=0.1, 
                 binary=True, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional

        self.image_gaussian_noise = GaussianNoise(0.5)
        self.conv3d_nets = []
        for i in range(len(filters_3d)):
            conv3d_net = Sequential([
                Conv3D(filters_3d[i], kernel_size3d, padding="same", use_bias=False),
                BatchNormalization(),
                ReLU(),
                MaxPool3D(pool_size=(2, 2, 2), strides=(2, 1, 1), padding="same"),
                Dropout(dropout)
            ], name=f"conv3d_net_{i}")
            self.conv3d_nets.append(conv3d_net)
        self.global_maxpool = GlobalMaxPool3D()

        self.conv1d_nets = []
        for i in range(len(filters_1d)):
            conv1d_net = Sequential([
                Conv1D(filters_1d[i], kernel_size1d, padding="same", kernel_initializer="he_normal", use_bias=False),
                BatchNormalization(),
                ELU(),
                AveragePooling1D(pool_size=2, strides=2, padding="same"),
                Dropout(dropout)
            ], name=f"conv1d_net_{i}")
            self.conv1d_nets.append(conv1d_net)

        self.masking = Masking(mask_value=0.0)
        self.time_series_gaussian_noise = GaussianNoise(0.5)
        self.rnns = []
        # self.skip_dense = Dense(rnn_hidden_size*2, name="skip_connection_dense_bidirectional")
        for i in range(num_rnn_layers):
            return_sequences = True if i!=num_rnn_layers-1 else False
            if mode == "RNN":
                rnn = SimpleRNN(rnn_hidden_size, kernel_initializer="glorot_normal", recurrent_initializer="orthogonal", 
                                dropout=dropout, recurrent_dropout=dropout, return_sequences=return_sequences)
            elif mode == "LSTM":
                rnn = LSTM(rnn_hidden_size, kernel_initializer="glorot_normal", recurrent_initializer="orthogonal", 
                           dropout=dropout, recurrent_dropout=dropout, return_sequences=return_sequences)
            else:
                rnn = GRU(rnn_hidden_size, kernel_initializer="glorot_normal", recurrent_initializer="orthogonal", 
                          dropout=dropout, recurrent_dropout=dropout, return_sequences=return_sequences)
            if bidirectional:
                rnn = Bidirectional(rnn, name=f"{mode}_{i}")
            self.rnns.append(rnn)
        
        # self.dense_flat = Sequential([
        #     Dense(1, activation="relu"),
        #     Flatten()
        # ], name="dense_and_flatten")

        if regularizer == "l1":
            self.dense0 = Dense(hidden_size, activation="relu", 
                                kernel_regularizer=L1(l1_penalty),
                                bias_regularizer=L1(l1_penalty))
            self.dense1_0 = Dense(1, activation="sigmoid",
                                  kernel_regularizer=L1(l1_penalty),
                                  bias_regularizer=L1(l1_penalty))
            self.dense1_1 = Dense(18, activation="softmax", kernel_initializer="glorot_normal",
                                  kernel_regularizer=L1(l1_penalty),
                                  bias_regularizer=L1(l1_penalty))
        elif regularizer == "l2":
            self.dense0 = Dense(hidden_size, activation="relu", 
                                kernel_regularizer=L2(l2_penalty),
                                bias_regularizer=L2(l2_penalty))
            self.dense1_0 = Dense(1, activation="sigmoid", 
                                  kernel_regularizer=L2(l2_penalty),
                                  bias_regularizer=L2(l2_penalty))
            self.dense1_1 = Dense(18, activation="softmax", kernel_initializer="glorot_normal",
                                  kernel_regularizer=L2(l2_penalty),
                                  bias_regularizer=L2(l2_penalty))
        elif regularizer == "l1l2":
            self.dense0 = Dense(hidden_size, activation="relu", 
                                kernel_regularizer=L1L2(l1_penalty, l2_penalty),
                                bias_regularizer=L1L2(l1_penalty, l2_penalty))
            self.dense1_0 = Dense(1, activation="sigmoid", 
                                  kernel_regularizer=L1L2(l1_penalty, l2_penalty),
                                  bias_regularizer=L1L2(l1_penalty, l2_penalty))
            self.dense1_1 = Dense(18, activation="softmax", kernel_initializer="glorot_normal",
                                  kernel_regularizer=L1L2(l1_penalty, l2_penalty),
                                  bias_regularizer=L1L2(l1_penalty, l2_penalty))
        else:
            self.dense0 = Dense(hidden_size, activation="relu")
            self.dense1_0 = Dense(1, activation="sigmoid")
            self.dense1_1 = Dense(18, activation="softmax")

        self.binary = binary

    def build(self, input_shapes, training=False):
        image_shape = input_shapes[0]
        # self.image_gaussian_noise.build(image_shape)
        # image_shape = self.image_gaussian_noise.compute_output_shape(image_shape)
        for i in range(len(self.conv3d_nets)):
            self.conv3d_nets[i].build(image_shape)
            image_shape = self.conv3d_nets[i].compute_output_shape(image_shape)
        self.global_maxpool.build(image_shape)
        image_shape = self.global_maxpool.compute_output_shape(image_shape)

        time_series_shape = input_shapes[1]
        for i in range(len(self.conv1d_nets)):
            self.conv1d_nets[i].build(time_series_shape)
            shape = self.conv1d_nets[i].compute_output_shape(time_series_shape)

        self.masking.build(time_series_shape)
        time_series_shape = self.masking.compute_output_shape(time_series_shape)
        # self.time_series_gaussian_noise.build(time_series_shape)
        # image_shape = self.time_series_gaussian_noise.compute_output_shape(time_series_shape)
        # self.skip_dense.build(shape)
        for i in range(len(self.rnns)):
            self.rnns[i].build(time_series_shape)
            time_series_shape = self.rnns[i].compute_output_shape(time_series_shape)
        
        shape = (image_shape[0], image_shape[1] + time_series_shape[1] + input_shapes[2][1])

        self.dense0.build(shape)
        shape = self.dense0.compute_output_shape(shape)

        self.dense1_0.build(shape)
        self.dense1_1.build(shape)
        

    def call(self, inputs, training=False):
        image_out = inputs[0]
        image_out = self.image_gaussian_noise(image_out)
        for i in range(len(self.conv3d_nets)):
            image_out = self.conv3d_nets[i](image_out, training=training)
        image_out = self.global_maxpool(image_out)   # (batch, filters)

        time_series_out = inputs[1]
        # out = tf.RaggedTensor.from_tensor(out, lengths=inputs[3])
        for i in range(len(self.conv1d_nets)):
            time_series_out = self.conv1d_nets[i](time_series_out, training=training)

        time_series_out = self.masking(time_series_out)
        mask = self.masking.compute_mask(time_series_out)
        time_series_out = self.time_series_gaussian_noise(time_series_out)
        # time_series_out = tf.ragged.boolean_mask(time_series_out, mask)
        for i in range(len(self.rnns)):
            time_series_out = self.rnns[i](time_series_out, mask=mask, training=training)
            # if i==0 and len(self.rnns)!=1:
            #     out = self.skip_dense(out) if self.bidirectional else out
            #     out = rnn_out + out
            # if i < len(self.rnns)-1:
            #     out = rnn_out + out
            # else:
            #     out = rnn_out
        # out = self.rnn(out, training=training)        # (batch, downsampled_timestep, rnn_hidden_size)
        out = tf.concat([image_out, time_series_out, inputs[2]], axis=-1)
        out = self.dense0(out)
        if self.binary:
            out = self.dense1_0(out)
        else:
            out = self.dense1_1(out)
        return out

    def set_binary(self):
        self.binary = True
        if self.dense1_0.build == False:
            self.dense1_0.build(input_shape=(None, self.hidden_size))

    def set_multi(self):
        self.binary = False
        if self.dense1_1.build == False:
            self.dense1_1.build(input_shape=(None, self.hidden_size))

    def freeze_conv_timeseries(self):
        for i in range(len(self.conv2d_nets)):
            self.conv3d_nets[i].trainable = False
        for i in range(len(self.conv1d_nets)):
            self.conv1d_nets[i].trainable = False
        self.rnn.trainable = False

In [6]:
for i, (inputs, targets) in enumerate(type_train_ds):
    if i == 0:
        print(inputs[0].shape)
        print(inputs[1].shape)
        print(inputs[2].shape)
        break

(64, 50, 8, 8, 5)
(64, 50, 19)
(64, 17)


In [7]:
import json

with open('/kaggle/input/cmi-tf-datasets/sample_weight.json') as f:
    sample_weight_dict = json.load(f)

type_sample_weight = sample_weight_dict["type_sample_weight"]
type_class_weight = {0: type_sample_weight[0], 1: type_sample_weight[1]}

gesture_sample_weight = sample_weight_dict["gesture_sample_weight"]
gesture_class_weight = {i: gesture_sample_weight[i] for i in range(len(gesture_sample_weight))}

In [8]:
def add_type_sample_weight(x, y):
    y_int = tf.cast(y, tf.int32)
    weight = tf.gather([type_class_weight[0], type_class_weight[1]], y_int)
    return x, y, weight

def add_gesture_sample_weight(x, y):
    y_arg = tf.argmax(y)
    weight = tf.gather(gesture_sample_weight, y_arg)
    return x, y, weight

In [9]:
type_train_ds = type_train_ds.unbatch().map(add_type_sample_weight).batch(32, drop_remainder=True)
type_valid_ds = type_valid_ds.unbatch().map(add_type_sample_weight).batch(32, drop_remainder=True)

In [10]:
# gesture_train_ds = gesture_train_ds.unbatch().map(add_gesture_sample_weight).batch(16)
# gesture_valid_ds = gesture_valid_ds.unbatch().map(add_gesture_sample_weight).batch(16)
gesture_train_ds = gesture_train_ds.unbatch().batch(256)
gesture_valid_ds = gesture_valid_ds.unbatch().batch(256)

In [11]:
def scheduler(epoch):
    lr = 0.001
    if epoch < 20:
        return lr
    elif 20 <= epoch and epoch < 40:
        return lr * (1/2)
    elif 40 <= epoch and epoch < 60:
        return lr * (1/4)
    elif 60 <= epoch and epoch < 80:
        return lr * (1/8)
    else:
        return lr * (1/16)

In [12]:
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, CategoricalFocalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import BinaryAccuracy, F1Score, AUC

model = RNNModel(kernel_size3d=(5, 3, 3), kernel_size1d=5, filters_3d=[8, 16, 32], dropout=0.2,
                 filters_1d=[], num_rnn_layers=1, rnn_hidden_size=128, mode="LSTM", 
                 bidirectional=False, hidden_size=64, regularizer="l1l2", 
                 l1_penalty=1e-6, l2_penalty=1e-6, binary=False)
model.build(input_shapes=((None, None, 8, 8, 5), (None, None, 19), (None, 17)))
# model.set_multi()
model.compile(loss=CategoricalFocalCrossentropy(alpha=gesture_sample_weight, 
                                                gamma=2,
                                                label_smoothing=0.1),
              optimizer=Adam(learning_rate=5e-3),
              metrics=["accuracy"])
model.summary()
# model.freeze_conv_timeseries()

Model: "rnn_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gaussian_noise (GaussianNoise)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_net_0 (Sequential)       │ (None, None, 8, 8, 8)  │         1,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_net_1 (Sequential)       │ (None, None, 8, 8, 16) │         5,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_net_2 (Sequential)       │ (None, None, 8, 8, 32) │        23,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling3d            │ (None, 32)             │             0 │
│ (GlobalMaxPooling3D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masking (Masking)               │ (None, None, 19)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise_1                │ ?                      │             0 │
│ (GaussianNoise)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │        75,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        11,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 18)             │         1,170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,227 (465.73 KB)

 Trainable params: 119,115 (465.29 KB)

 Non-trainable params: 112 (448.00 B)

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
checkpoint_filepath = "/kaggle/working/cmi_best_model.weights.h5"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
    mode="max",
    verbose=1
)

lr_callback = LearningRateScheduler(scheduler, verbose=0)

history = model.fit(gesture_train_ds, epochs=200, 
                    validation_data=gesture_valid_ds,
                    callbacks=[checkpoint_callback])
model.load_weights(checkpoint_filepath)

Epoch 1/200


I0000 00:00:1755063373.062114      58 cuda_dnn.cc:529] Loaded cuDNN version 90300


     37/Unknown 18s 161ms/step - accuracy: 0.1530 - loss: 6.6275
Epoch 1: val_accuracy improved from -inf to 0.14022, saving model to /kaggle/working/cmi_best_model.weights.h5
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 380ms/step - accuracy: 0.1542 - loss: 6.6060 - val_accuracy: 0.1402 - val_loss: 6.9727
Epoch 2/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.2967 - loss: 4.9371
Epoch 2: val_accuracy improved from 0.14022 to 0.30349, saving model to /kaggle/working/cmi_best_model.weights.h5
37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 246ms/step - accuracy: 0.2972 - loss: 4.9342 - val_accuracy: 0.3035 - val_loss: 5.3557
Epoch 3/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.3545 - loss: 4.4795
Epoch 3: val_accuracy improved from 0.30349 to 0.34694, saving model to /kaggle/working/cmi_best_model.weights.h5
37/37 ━━━━━━━━━━━━━━━━━━━━ 9s 244ms/step - accuracy: 0.3548 - loss: 4.4779 - val_accuracy: 0.3469 - val_loss: 4.6575
Epoch 4/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.386

In [14]:
import json

with open('/kaggle/input/cmi-tf-datasets/mapping.json') as f:
    mapping_dict = json.load(f)

gesture_mapping = mapping_dict["gesture_mapping"]
inv_gesture_mapping = {value: key for key, value in gesture_mapping.items()}

num2gesture = np.vectorize(lambda x: inv_gesture_mapping[x])

In [15]:
non_target_gestures = ["Drink from bottle/cup", "Glasses on/off", "Pull air toward your face",
                       "Pinch knee/leg skin", "Scratch knee/leg skin", "Write name on leg",
                       "Text on phone", "Feel around in tray and pull out an object",
                       "Write name in air", "Wave hello"]

def map_non_target(y_ind):
    y_pred = inv_gesture_mapping[y_ind]
    if y_ind == 3:
        y_ind = 2
    elif y_ind == 4:
        y_ind = 3
    elif y_ind == 6:
        y_ind = 4
    elif y_ind == 7:
        y_ind = 5
    elif y_ind == 9:
        y_ind = 6
    elif y_ind == 10:
        y_ind = 7
    if y_pred in non_target_gestures:
        y_ind = 8
    return y_ind

vectorize_map_non_target = np.vectorize(map_non_target)

In [16]:
conf_tensor = np.zeros((9, 9), dtype=np.int32)
for i, (inputs, labels) in enumerate(gesture_valid_ds):
    labels_pred = model.predict(inputs, verbose=0)
    labels_pred = tf.argmax(labels_pred, axis=-1).numpy()
    labels_pred = vectorize_map_non_target(labels_pred)
    labels_true = tf.argmax(labels, axis=-1).numpy()
    labels_true = vectorize_map_non_target(labels_true)
    conf_tensor += tf.math.confusion_matrix(labels_true, labels_pred, num_classes=9)

In [17]:
precisions = []
recalls = []
f1s = []

for i in range(9):
    column = conf_tensor[i, :]
    row = conf_tensor[:, i]
    precision = column[i] / tf.math.reduce_sum(column)
    recall = row[i] / tf.math.reduce_sum(row)
    inv_f1 = (1/precision + 1/recall)/2
    f1 = 1/inv_f1
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
print(f"F1 score mean: {np.round(np.mean(f1s), 3)}")

F1 score mean: 0.512


In [18]:
# import os
# save_path = os.path.join("/kaggle/working/", "cmi_model.weights.h5")
# model.save_weights(save_path)

In [19]:
new_model = RNNModel(kernel_size3d=(5, 3, 3), kernel_size1d=5, filters_3d=[8, 16, 32], dropout=0.2,
                     filters_1d=[], num_rnn_layers=1, rnn_hidden_size=128, mode="LSTM", 
                     bidirectional=False, hidden_size=64, regularizer="l1l2", 
                     l1_penalty=1e-6, l2_penalty=1e-6, binary=False)
new_model.build(input_shapes=((None, None, 8, 8, 5), (None, None, 19), (None, 17)))
new_model.load_weights(checkpoint_filepath)
new_model.summary()

Model: "rnn_model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gaussian_noise_2                │ ?                      │             0 │
│ (GaussianNoise)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_net_0 (Sequential)       │ (None, None, 8, 8, 8)  │         1,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_net_1 (Sequential)       │ (None, None, 8, 8, 16) │         5,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_net_2 (Sequential)       │ (None, None, 8, 8, 32) │        23,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling3d_1          │ (None, 32)             │             0 │
│ (GlobalMaxPooling3D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masking_1 (Masking)             │ (None, None, 19)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise_3                │ ?                      │             0 │
│ (GaussianNoise)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        75,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        11,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 18)             │         1,170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,227 (465.73 KB)

 Trainable params: 119,115 (465.29 KB)

 Non-trainable params: 112 (448.00 B)